In [1]:
!pip install -qq torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -qq ensemble-boxes

# Install mmcv
!pip install -qq mmcv-full==1.6.0 -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

    
# Install mmdetection
!git clone https://github.com/alvaromoure/mmdetection.git
%cd mmdetection
!pip install -e .


# Install YoloV5
%cd /kaggle/working
!git clone https://github.com/alvaromoure/yolov5.git
%cd yolov5
%pip install -qr requirements.txt  # install dependencies
%cd /kaggle/working
%cd mmdetection

Cloning into 'mmdetection'...
remote: Enumerating objects: 24866, done.
remote: Total 24866 (delta 0), reused 0 (delta 0), pack-reused 24866
Receiving objects: 100% (24866/24866), 37.84 MiB | 22.39 MiB/s, done.
Resolving deltas: 100% (17450/17450), done.
/kaggle/working/mmdetection
Obtaining file:///kaggle/working/mmdetection
  Preparing metadata (setup.py) ... - \ done
  Installing build dependencies ... - \ | / - \ | / - \ done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
  Created wheel for pycocotools: filename=pycocotools-2.0.5-cp37-cp37m-linux_x86_64.whl size=373765 sha256=a9d8cdc9028fd77ae72c6abba1ce47147e092aba13bd8b0c68248e144971ecdf
  Stored in directory: /root/.cache/pip/wheels/85/c4/f0/7128093a134f590e4383fd60cb484960878721d98b9a515317
Successfully built pycocotools
  Running setup.py develop for mmdet
/kaggle/working
Cloning into 'yolov5'...
remote: Enumerating objects: 10481, done.
remote: To

# Aux Functions

In [2]:
def get_bbox(row):
    bboxes = []
    bbox = []
    for i, l in enumerate(row.label.split(' ')):
        if (i % 6 == 0) | (i % 6 == 1):
            continue
        bbox.append(float(l))
        if i % 6 == 5:
            bboxes.append(bbox)
            bbox = []  
            
    return bboxes

# Scale the bounding boxes according to the size of the resized image. 
def scale_bbox(row, bboxes):
    # Get scaling factor
    scale_x = IMG_SIZE/row.dim1
    scale_y = IMG_SIZE/row.dim0
    
    scaled_bboxes = []
    for bbox in bboxes:
        x = int(np.round(bbox[0]*scale_x, 4))
        y = int(np.round(bbox[1]*scale_y, 4))
        x1 = int(np.round(bbox[2]*(scale_x), 4))
        y1= int(np.round(bbox[3]*scale_y, 4))

        scaled_bboxes.append([x, y, x1, y1]) # xmin, ymin, xmax, ymax
        
    return scaled_bboxes

# Convert the bounding boxes in YOLO format.
def get_yolo_format_bbox(img_w, img_h, bboxes):
    yolo_boxes = []
    for bbox in bboxes:
        w = bbox[2] - bbox[0] # xmax - xmin
        h = bbox[3] - bbox[1] # ymax - ymin
        xc = bbox[0] + int(np.round(w/2)) # xmin + width/2
        yc = bbox[1] + int(np.round(h/2)) # ymin + height/2
        
        yolo_boxes.append([xc/img_w, yc/img_h, w/img_w, h/img_h]) # x_center y_center width height
    
    return yolo_boxes

def yolo_to_coco_bbox(img_size,bbox):
    w = bbox[2]*img_size
    h = bbox[3]*img_size
    x1 = bbox[0]*img_size - w/2
    y1 = bbox[1]*img_size - h/2
    return [x1,y1,w,h]

def get_bboxes_from_result(result,img_size):
    boxes_list = []
    for array in tqdm(result,total=len(result)):
        for ann in array[0].tolist():
            x1,y1,x2,y2 = [float(coord) for coord in ann[:4]]            
            boxes_list.append([x1/img_size, y1/img_size, x2/img_size , y2/img_size])
    return boxes_list
        
def get_scores_from_result(result):
    scores_list = []
    for array in tqdm(result,total=len(result)):
        for ann in array[0].tolist():
            score = float(ann[4])
            scores_list.append(score)
    return scores_list


def normalize_bbox(bbox, w, h):
    x1, y1, x2, y2 = [float(num) for num in bbox]
    return [x1/w, y1/h, x2/w, y2/h]

def resize_bbox(bbox, w, h):
    x1, y1, x2, y2 = [float(num) for num in bbox]
    return [x1*w, y1*h, x2*w, y2*h]

def results_to_json(results):
    out_results_list = []
    for img_id,res_array in tqdm(enumerate(results)):
        for ann in res_array[0]:
            coords = ann[:4].tolist()
            ann_dict = dict(
                image_id = int(img_id),
                bbox = [ coords[0], coords[1], coords[2]-coords[0] , coords[3]-coords[1] ],
                score= float(ann[4]),
                category_id = 0
            )
            out_results_list.append(ann_dict)
    return out_results_list


# Imports

In [3]:
import wandb
import os
import shutil
import sys
import json
import glob
import pickle
import yaml

from tqdm import tqdm
import torch,torchvision
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from collections import defaultdict
from datetime import datetime
from mmcv import Config

import mmdet
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.apis import init_detector, inference_detector, show_result_pyplot

from ensemble_boxes import *

import random
import numpy as np
from pathlib import Path
from pycocotools import cocoeval
from pycocotools.cocoeval import COCOeval
from pycocotools.coco import COCO

import IPython.display as ipd
import numpy as np
def boom():
    beep = np.sin(2*np.pi*400*np.arange(10000*3)/10000)
    return ipd.Audio(beep, rate=10000, autoplay=True)

In [4]:
seed = 42
set_random_seed(seed, deterministic=False)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ['PYTHONHASHSEED'] = str(seed)

In [5]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
wandb_api = user_secrets.get_secret("wandb-key") 
wandb.login(key=wandb_api,relogin=True)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
PROJECT_NAME = 'Final-Covid19-Detection'
NOTEBOOK_NAME = 'TFM - Inference Firefox'
EXP_NAME = 'Inference'

In [7]:
import pytz
timezone = pytz.timezone('Europe/Madrid')
now = datetime.now(timezone)

dt_string = now.strftime("%d/%m_%H:%M")

run = wandb.init(project=PROJECT_NAME,
                 name=f'{EXP_NAME}_{dt_string}',
                job_type='inference',
                notes=f'Notebook: {NOTEBOOK_NAME}')

wandb: Currently logged in as: alvaromoureupm. Use `wandb login --relogin` to force relogin


## Geting Validation Images

In [8]:
val_ann_file_0 = '/kaggle/working/coco_val_512x512_fold_0.json'
val_ann_file_1 = '/kaggle/working/coco_val_512x512_fold_1.json'
val_ann_file_2 = '/kaggle/working/coco_val_512x512_fold_2.json'
val_ann_file_3 = '/kaggle/working/coco_val_512x512_fold_3.json'
val_ann_file_4 = '/kaggle/working/coco_val_512x512_fold_4.json'

In [9]:
def download_wandb_models(run,model_name,model_list):
    downloaded_models = []
    os.makedirs('/kaggle/tmp/downloaded_models',exist_ok=True)
    for i,model in enumerate(tqdm(model_list)):
        os.makedirs(f'/kaggle/tmp/downloaded_models/{model_name}',exist_ok=True)
        artifact = run.use_artifact(f'alvaromoureupm/Final-Covid19-Detection/{model}')
        download_dir = artifact.download()
        model_files = os.listdir(download_dir)
        for file in model_files:
            if (file.endswith('.pth') or file.endswith('.pt')):
                file_name = f'fold_{i}_{file}'
            else:
                file_name = file
            shutil.copyfile(os.path.join(download_dir,file),f'/kaggle/tmp/downloaded_models/{model_name}/{file_name}')
            if (file.endswith('.pth') or file.endswith('.pt')):
                downloaded_models.append(file_name)
    return downloaded_models

# MMDET Models

# RetinaNet

In [10]:
downloaded_retinanet = download_wandb_models(run,'RetinaNet',['RetinaNet_Fold_0:latest',
                                                              'RetinaNet_Fold_1:latest',
                                                             'RetinaNet_Fold_2:latest',
                                                             'RetinaNet_Fold_3:latest',
                                                             'RetinaNet_Fold_4:latest'])

retinanet_checkpoint_0 = f'/kaggle/tmp/downloaded_models/RetinaNet/{downloaded_retinanet[0]}'
retinanet_checkpoint_1 = f'/kaggle/tmp/downloaded_models/RetinaNet/{downloaded_retinanet[1]}'
retinanet_checkpoint_2 = f'/kaggle/tmp/downloaded_models/RetinaNet/{downloaded_retinanet[2]}'
retinanet_checkpoint_3 = f'/kaggle/tmp/downloaded_models/RetinaNet/{downloaded_retinanet[3]}'
retinanet_checkpoint_4 = f'/kaggle/tmp/downloaded_models/RetinaNet/{downloaded_retinanet[4]}'

100%|██████████| 5/5 [02:18<00:00, 27.70s/it]


In [11]:
common_config = '/kaggle/tmp/downloaded_models/RetinaNet/retinanet.py'

retina_cfg_0 = Config.fromfile(common_config)
retina_cfg_0.data.test = retina_cfg_0.data.val
retina_cfg_0.data.test.img_prefix = '/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/train_512x512'
retina_cfg_0.data.test.ann_file = val_ann_file_0
retina_cfg_0_file = '/kaggle/tmp/downloaded_models/RetinaNet/retina_cfg_0.py'
retina_cfg_0.dump(retina_cfg_0_file)

retina_cfg_1 = Config.fromfile(common_config)
retina_cfg_1.data.test = retina_cfg_1.data.val
retina_cfg_1.data.test.img_prefix = '/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/train_512x512'
retina_cfg_1.data.test.ann_file = val_ann_file_1
retina_cfg_1_file = '/kaggle/tmp/downloaded_models/RetinaNet/retina_cfg_1.py'
retina_cfg_1.dump(retina_cfg_1_file)

retina_cfg_2 = Config.fromfile(common_config)
retina_cfg_2.data.test = retina_cfg_2.data.val
retina_cfg_2.data.test.img_prefix = '/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/train_512x512'
retina_cfg_2.data.test.ann_file = val_ann_file_2
retina_cfg_2_file = '/kaggle/tmp/downloaded_models/RetinaNet/retina_cfg_2.py'
retina_cfg_2.dump(retina_cfg_2_file)

retina_cfg_3 = Config.fromfile(common_config)
retina_cfg_3.data.test = retina_cfg_3.data.val
retina_cfg_3.data.test.img_prefix = '/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/train_512x512'
retina_cfg_3.data.test.ann_file = val_ann_file_3
retina_cfg_3_file = '/kaggle/tmp/downloaded_models/RetinaNet/retina_cfg_3.py'
retina_cfg_3.dump(retina_cfg_3_file)

retina_cfg_4 = Config.fromfile(common_config)
retina_cfg_4.data.test = retina_cfg_4.data.val
retina_cfg_4.data.test.img_prefix = '/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/train_512x512'
retina_cfg_4.data.test.ann_file = val_ann_file_4
retina_cfg_4_file = '/kaggle/tmp/downloaded_models/RetinaNet/retina_cfg_4.py'
retina_cfg_4.dump(retina_cfg_4_file)

retinanet_0 = init_detector(retina_cfg_0_file,retinanet_checkpoint_0,device='cuda:0')
retinanet_0.CLASSES = ('Covid_Opacity',)
retinanet_1 = init_detector(retina_cfg_1_file,retinanet_checkpoint_1,device='cuda:0')
retinanet_1.CLASSES = ('Covid_Opacity',)
retinanet_2 = init_detector(retina_cfg_2_file,retinanet_checkpoint_2,device='cuda:0')
retinanet_2.CLASSES = ('Covid_Opacity',)
retinanet_3 = init_detector(retina_cfg_3_file,retinanet_checkpoint_3,device='cuda:0')
retinanet_3.CLASSES = ('Covid_Opacity',)
retinanet_4 = init_detector(retina_cfg_4_file,retinanet_checkpoint_4,device='cuda:0')
retinanet_4.CLASSES = ('Covid_Opacity',)

In [12]:
!python tools/test.py {retina_cfg_0_file} {retinanet_checkpoint_0}\
--out /kaggle/working/results/retinanet/inference_results_0.pkl\
--eval bbox\
--show-dir /kaggle/working/results/retinanet/result_images_0\
--show-score-thr 0.3 \
--eval-options "jsonfile_prefix=/kaggle/working/results/retinanet/retinanet_0"

!python tools/test.py {retina_cfg_1_file} {retinanet_checkpoint_1}\
--out /kaggle/working/results/retinanet/inference_results_1.pkl\
--eval bbox\
--show-dir /kaggle/working/results/retinanet/result_images_1\
--show-score-thr 0.3 \
--eval-options "jsonfile_prefix=/kaggle/working/results/retinanet/retinanet_1"

!python tools/test.py {retina_cfg_2_file} {retinanet_checkpoint_2}\
--out /kaggle/working/results/retinanet/inference_results_2.pkl\
--eval bbox\
--show-dir /kaggle/working/results/retinanet/result_images_2\
--show-score-thr 0.3 \
--eval-options "jsonfile_prefix=/kaggle/working/results/retinanet/retinanet_2"

!python tools/test.py {retina_cfg_3_file} {retinanet_checkpoint_3}\
--out /kaggle/working/results/retinanet/inference_results_3.pkl\
--eval bbox\
--show-dir /kaggle/working/results/retinanet/result_images_3\
--show-score-thr 0.3 \
--eval-options "jsonfile_prefix=/kaggle/working/results/retinanet/retinanet_3"

!python tools/test.py {retina_cfg_4_file} {retinanet_checkpoint_4}\
--out /kaggle/working/results/retinanet/inference_results_4.pkl\
--eval bbox\
--show-dir /kaggle/working/results/retinanet/result_images_4\
--show-score-thr 0.3 \
--eval-options "jsonfile_prefix=/kaggle/working/results/retinanet/retinanet_4"

/kaggle/working/mmdetection/mmdet/utils/setup_env.py:39: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting OMP_NUM_THREADS environment variable for each process '
/kaggle/working/mmdetection/mmdet/utils/setup_env.py:49: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting MKL_NUM_THREADS environment variable for each process '
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
load checkpoint from local path: /kaggle/tmp/downloaded_models/RetinaNet/fold_0_best_bbox_mAP_50_epoch_16.pth
[                                                  ] 0/859, elapsed: 0s, ETA:/opt/conda/lib/python3.7/site-packag

### 5-fold mAP@50 computation

In [13]:
!ls /kaggle/working/results/retinanet

inference_results_0.pkl  result_images_0  retinanet_0.bbox.json
inference_results_1.pkl  result_images_1  retinanet_1.bbox.json
inference_results_2.pkl  result_images_2  retinanet_2.bbox.json
inference_results_3.pkl  result_images_3  retinanet_3.bbox.json
inference_results_4.pkl  result_images_4  retinanet_4.bbox.json


In [14]:
retina_mAP50_list = []
for fold in range(5):
    cocoGt = COCO(f'/kaggle/working/coco_val_512x512_fold_{fold}.json')
    cocoDt = cocoGt.loadRes(f'/kaggle/working/results/retinanet/retinanet_{fold}.bbox.json')
    imgIds=sorted(cocoGt.getImgIds())
    cocoEval = COCOeval(cocoGt,cocoDt,'bbox')

    cocoEval.params.catIds = [0] 
    cocoEval.evaluate()  
    cocoEval.accumulate()
    cocoEval.summarize()
    mAP50 = cocoEval.stats[1]
    retina_mAP50_list.append(mAP50)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.82s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=5.32s).
Accumulating evaluation results...
DONE (t=0.88s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.171
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.545
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.053
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.001
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.054
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.197
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.131
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.341
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

## RetinaNet Accuracy

In [15]:
np.mean(retina_mAP50_list)

0.5254761053377951

# Faster R-CNN

In [16]:
downloaded_faster_rcnn = download_wandb_models(run,'Faster_RCNN',['faster_rcnn_Fold0:v1','faster_rcnn_Fold1:v0',
                                                                    'faster_rcnn_Fold2:v0','faster_rcnn_Fold3:v0',
                                                                   'faster_rcnn_Fold4:v0'])

faster_checkpoint_0 = f'/kaggle/tmp/downloaded_models/Faster_RCNN/{downloaded_faster_rcnn[0]}'
faster_checkpoint_1 = f'/kaggle/tmp/downloaded_models/Faster_RCNN/{downloaded_faster_rcnn[1]}'
faster_checkpoint_2 = f'/kaggle/tmp/downloaded_models/Faster_RCNN/{downloaded_faster_rcnn[2]}'
faster_checkpoint_3 = f'/kaggle/tmp/downloaded_models/Faster_RCNN/{downloaded_faster_rcnn[3]}'
faster_checkpoint_4 = f'/kaggle/tmp/downloaded_models/Faster_RCNN/{downloaded_faster_rcnn[4]}'

100%|██████████| 5/5 [02:30<00:00, 30.00s/it]


In [17]:
!ls /kaggle/working/downloaded_models/Faster_RCNN

ls: cannot access '/kaggle/working/downloaded_models/Faster_RCNN': No such file or directory


In [18]:
common_config = '/kaggle/tmp/downloaded_models/Faster_RCNN/faster_rcnn_x101_32x4d_fpn_1x_coco_faster_rcnn_Fold0.py'

faster_cfg_0 = Config.fromfile(common_config)
faster_cfg_0.data.test = faster_cfg_0.data.val
faster_cfg_0.data.test.img_prefix = '/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/train_512x512'
faster_cfg_0.data.test.ann_file = val_ann_file_0
faster_cfg_0_file = '/kaggle/tmp/downloaded_models/Faster_RCNN/faster_cfg_0.py'
faster_cfg_0.dump(faster_cfg_0_file)

faster_cfg_1 = Config.fromfile(common_config)
faster_cfg_1.data.test = faster_cfg_1.data.val
faster_cfg_1.data.test.img_prefix = '/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/train_512x512'
faster_cfg_1.data.test.ann_file = val_ann_file_1
faster_cfg_1_file = '/kaggle/tmp/downloaded_models/Faster_RCNN/faster_cfg_1.py'
faster_cfg_1.dump(faster_cfg_1_file)

faster_cfg_2 = Config.fromfile(common_config)
faster_cfg_2.data.test = faster_cfg_2.data.val
faster_cfg_2.data.test.img_prefix = '/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/train_512x512'
faster_cfg_2.data.test.ann_file = val_ann_file_2
faster_cfg_2_file = '/kaggle/tmp/downloaded_models/Faster_RCNN/faster_cfg_2.py'
faster_cfg_2.dump(faster_cfg_2_file)

faster_cfg_3 = Config.fromfile(common_config)
faster_cfg_3.data.test = faster_cfg_3.data.val
faster_cfg_3.data.test.img_prefix = '/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/train_512x512'
faster_cfg_3.data.test.ann_file = val_ann_file_3
faster_cfg_3_file = '/kaggle/tmp/downloaded_models/Faster_RCNN/faster_cfg_3.py'
faster_cfg_3.dump(faster_cfg_3_file)

faster_cfg_4 = Config.fromfile(common_config)
faster_cfg_4.data.test = faster_cfg_4.data.val
faster_cfg_4.data.test.img_prefix = '/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/train_512x512'
faster_cfg_4.data.test.ann_file = val_ann_file_4
faster_cfg_4_file = '/kaggle/tmp/downloaded_models/Faster_RCNN/faster_cfg_4.py'
faster_cfg_4.dump(faster_cfg_4_file)

In [19]:
!python tools/test.py {faster_cfg_0_file} {faster_checkpoint_0}\
--out /kaggle/working/results/faster_rcnn/inference_results_0.pkl\
--eval bbox\
--show-dir /kaggle/working/results/faster_rcnn/result_images_0\
--show-score-thr 0.3 \
--eval-options "jsonfile_prefix=/kaggle/working/results/faster_rcnn/faster_0"

!python tools/test.py {faster_cfg_1_file} {faster_checkpoint_1}\
--out /kaggle/working/results/faster_rcnn/inference_results_1.pkl\
--eval bbox\
--show-dir /kaggle/working/results/faster_rcnn/result_images_1\
--show-score-thr 0.3 \
--eval-options "jsonfile_prefix=/kaggle/working/results/faster_rcnn/faster_1"

!python tools/test.py {faster_cfg_2_file} {faster_checkpoint_2}\
--out /kaggle/working/results/faster_rcnn/inference_results_2.pkl\
--eval bbox\
--show-dir /kaggle/working/results/faster_rcnn/result_images_2\
--show-score-thr 0.3 \
--eval-options "jsonfile_prefix=/kaggle/working/results/faster_rcnn/faster_2"

!python tools/test.py {faster_cfg_3_file} {faster_checkpoint_3}\
--out /kaggle/working/results/faster_rcnn/inference_results_3.pkl\
--eval bbox\
--show-dir /kaggle/working/results/faster_rcnn/result_images_3\
--show-score-thr 0.3 \
--eval-options "jsonfile_prefix=/kaggle/working/results/faster_rcnn/faster_3"

!python tools/test.py {faster_cfg_4_file} {faster_checkpoint_4}\
--out /kaggle/working/results/faster_rcnn/inference_results_4.pkl\
--eval bbox\
--show-dir /kaggle/working/results/faster_rcnn/result_images_4\
--show-score-thr 0.3 \
--eval-options "jsonfile_prefix=/kaggle/working/results/faster_rcnn/faster_4"

/kaggle/working/mmdetection/mmdet/utils/setup_env.py:39: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting OMP_NUM_THREADS environment variable for each process '
/kaggle/working/mmdetection/mmdet/utils/setup_env.py:49: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting MKL_NUM_THREADS environment variable for each process '
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
load checkpoint from local path: /kaggle/tmp/downloaded_models/Faster_RCNN/fold_0_epoch_10.pth
[                                                  ] 0/859, elapsed: 0s, ETA:/opt/conda/lib/python3.7/site-packages/torch/nn/fun

In [20]:
!ls /kaggle/working/results/faster_rcnn

faster_0.bbox.json  inference_results_0.pkl  result_images_0
faster_1.bbox.json  inference_results_1.pkl  result_images_1
faster_2.bbox.json  inference_results_2.pkl  result_images_2
faster_3.bbox.json  inference_results_3.pkl  result_images_3
faster_4.bbox.json  inference_results_4.pkl  result_images_4


In [21]:
mAP50_list = []
for fold in range(5):
    cocoGt = COCO(f'/kaggle/working/coco_val_512x512_fold_{fold}.json')
    cocoDt = cocoGt.loadRes(f'/kaggle/working/results/faster_rcnn/faster_{fold}.bbox.json')
    imgIds=sorted(cocoGt.getImgIds())
    cocoEval = COCOeval(cocoGt,cocoDt,'bbox')

    cocoEval.params.catIds = [0] 
    cocoEval.evaluate()  
    cocoEval.accumulate()
    cocoEval.summarize()
    mAP50 = cocoEval.stats[1]
    mAP50_list.append(mAP50)

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.04s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.21s).
Accumulating evaluation results...
DONE (t=0.77s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.183
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.572
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.064
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.054
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.211
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.139
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.343
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

In [22]:
np.mean(mAP50_list)

0.5517452074712978

# Cascade R-CNN

In [23]:
downloaded_cascade_rcnn = download_wandb_models(run,'Cascade_RCNN',['cascade_rcnn_x101_Fold0:v0','cascade_rcnn_x101_Fold1:v0',
                                                                    'cascade_rcnn_x101_Fold2:v0','cascade_rcnn_x101_Fold3:v0',
                                                                   'cascade_rcnn_x101_Fold4:v0'])

cascade_checkpoint_0 = f'/kaggle/tmp/downloaded_models/Cascade_RCNN/{downloaded_cascade_rcnn[0]}'
cascade_checkpoint_1 = f'/kaggle/tmp/downloaded_models/Cascade_RCNN/{downloaded_cascade_rcnn[1]}'
cascade_checkpoint_2 = f'/kaggle/tmp/downloaded_models/Cascade_RCNN/{downloaded_cascade_rcnn[2]}'
cascade_checkpoint_3 = f'/kaggle/tmp/downloaded_models/Cascade_RCNN/{downloaded_cascade_rcnn[3]}'
cascade_checkpoint_4 = f'/kaggle/tmp/downloaded_models/Cascade_RCNN/{downloaded_cascade_rcnn[4]}'

100%|██████████| 5/5 [06:52<00:00, 82.46s/it]


In [24]:
!ls /kaggle/working/downloaded_models/Cascade_RCNN

ls: cannot access '/kaggle/working/downloaded_models/Cascade_RCNN': No such file or directory


In [25]:
common_config = '/kaggle/tmp/downloaded_models/Cascade_RCNN/cascade_rcnn_x101_32x4d_fpn_1x_coco_cascade_rcnn_x101_Fold0.py'

cascade_cfg_0 = Config.fromfile(common_config)
cascade_cfg_0.data.test = cascade_cfg_0.data.val
cascade_cfg_0.data.test.img_prefix = '/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/train_512x512'
cascade_cfg_0.data.test.ann_file = val_ann_file_0
cascade_cfg_0_file = '/kaggle/tmp/downloaded_models/Cascade_RCNN/cascade_cfg_0.py'
cascade_cfg_0.dump(cascade_cfg_0_file)

cascade_cfg_1 = Config.fromfile(common_config)
cascade_cfg_1.data.test = cascade_cfg_1.data.val
cascade_cfg_1.data.test.img_prefix = '/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/train_512x512'
cascade_cfg_1.data.test.ann_file = val_ann_file_1
cascade_cfg_1_file = '/kaggle/tmp/downloaded_models/Cascade_RCNN/cascade_cfg_1.py'
cascade_cfg_1.dump(cascade_cfg_1_file)

cascade_cfg_2 = Config.fromfile(common_config)
cascade_cfg_2.data.test = cascade_cfg_2.data.val
cascade_cfg_2.data.test.img_prefix = '/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/train_512x512'
cascade_cfg_2.data.test.ann_file = val_ann_file_2
cascade_cfg_2_file = '/kaggle/tmp/downloaded_models/Cascade_RCNN/cascade_cfg_2.py'
cascade_cfg_2.dump(cascade_cfg_2_file)

cascade_cfg_3 = Config.fromfile(common_config)
cascade_cfg_3.data.test = cascade_cfg_3.data.val
cascade_cfg_3.data.test.img_prefix = '/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/train_512x512'
cascade_cfg_3.data.test.ann_file = val_ann_file_3
cascade_cfg_3_file = '/kaggle/tmp/downloaded_models/Cascade_RCNN/cascade_cfg_3.py'
cascade_cfg_3.dump(cascade_cfg_3_file)

cascade_cfg_4 = Config.fromfile(common_config)
cascade_cfg_4.data.test = cascade_cfg_4.data.val
cascade_cfg_4.data.test.img_prefix = '/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/train_512x512'
cascade_cfg_4.data.test.ann_file = val_ann_file_4
cascade_cfg_4_file = '/kaggle/tmp/downloaded_models/Cascade_RCNN/cascade_cfg_4.py'
cascade_cfg_4.dump(cascade_cfg_4_file)


cascade_0 = init_detector(cascade_cfg_0_file,cascade_checkpoint_0,device='cuda:0')
cascade_0.CLASSES = ('Covid_Opacity',)

cascade_1 = init_detector(cascade_cfg_1_file,cascade_checkpoint_1,device='cuda:0')
cascade_1.CLASSES = ('Covid_Opacity',)

cascade_2 = init_detector(cascade_cfg_2_file,cascade_checkpoint_2,device='cuda:0')
cascade_2.CLASSES = ('Covid_Opacity',)

cascade_3 = init_detector(cascade_cfg_3_file,cascade_checkpoint_3,device='cuda:0')
cascade_3.CLASSES = ('Covid_Opacity',)

cascade_4 = init_detector(cascade_cfg_4_file,cascade_checkpoint_4,device='cuda:0')
cascade_4.CLASSES = ('Covid_Opacity',)

In [26]:
!python tools/test.py {cascade_cfg_0_file} {cascade_checkpoint_0}\
--out /kaggle/working/results/cascade_rcnn/inference_results_0.pkl\
--eval bbox\
--show-dir /kaggle/working/results/cascade_rcnn/result_images_0\
--show-score-thr 0.3 \
--eval-options "jsonfile_prefix=/kaggle/working/results/cascade_rcnn/cascade_0"

!python tools/test.py {cascade_cfg_1_file} {cascade_checkpoint_1}\
--out /kaggle/working/results/cascade_rcnn/inference_results_1.pkl\
--eval bbox\
--show-dir /kaggle/working/results/cascade_rcnn/result_images_1\
--show-score-thr 0.3 \
--eval-options "jsonfile_prefix=/kaggle/working/results/cascade_rcnn/cascade_1"

!python tools/test.py {cascade_cfg_2_file} {cascade_checkpoint_2}\
--out /kaggle/working/results/cascade_rcnn/inference_results_2.pkl\
--eval bbox\
--show-dir /kaggle/working/results/cascade_rcnn/result_images_2\
--show-score-thr 0.3 \
--eval-options "jsonfile_prefix=/kaggle/working/results/cascade_rcnn/cascade_2"

!python tools/test.py {cascade_cfg_3_file} {cascade_checkpoint_3}\
--out /kaggle/working/results/cascade_rcnn/inference_results_3.pkl\
--eval bbox\
--show-dir /kaggle/working/results/cascade_rcnn/result_images_3\
--show-score-thr 0.3 \
--eval-options "jsonfile_prefix=/kaggle/working/results/cascade_rcnn/cascade_3"

!python tools/test.py {cascade_cfg_4_file} {cascade_checkpoint_4}\
--out /kaggle/working/results/cascade_rcnn/inference_results_4.pkl\
--eval bbox\
--show-dir /kaggle/working/results/cascade_rcnn/result_images_4\
--show-score-thr 0.3 \
--eval-options "jsonfile_prefix=/kaggle/working/results/cascade_rcnn/cascade_4"


/kaggle/working/mmdetection/mmdet/utils/setup_env.py:39: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting OMP_NUM_THREADS environment variable for each process '
/kaggle/working/mmdetection/mmdet/utils/setup_env.py:49: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting MKL_NUM_THREADS environment variable for each process '
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
load checkpoint from local path: /kaggle/tmp/downloaded_models/Cascade_RCNN/fold_0_epoch_9.pth
[                                                  ] 0/859, elapsed: 0s, ETA:/opt/conda/lib/python3.7/site-packages/torch/nn/fun

In [27]:
!ls /kaggle/working/results/cascade_rcnn

cascade_0.bbox.json  inference_results_0.pkl  result_images_0
cascade_1.bbox.json  inference_results_1.pkl  result_images_1
cascade_2.bbox.json  inference_results_2.pkl  result_images_2
cascade_3.bbox.json  inference_results_3.pkl  result_images_3
cascade_4.bbox.json  inference_results_4.pkl  result_images_4


In [28]:
mAP50_list = []
for fold in range(5):
    cocoGt = COCO(f'/kaggle/working/coco_val_512x512_fold_{fold}.json')
    cocoDt = cocoGt.loadRes(f'/kaggle/working/results/cascade_rcnn/cascade_{fold}.bbox.json')
    imgIds=sorted(cocoGt.getImgIds())
    cocoEval = COCOeval(cocoGt,cocoDt,'bbox')

    cocoEval.params.catIds = [0] 
    cocoEval.evaluate()  
    cocoEval.accumulate()
    cocoEval.summarize()
    mAP50 = cocoEval.stats[1]
    mAP50_list.append(mAP50)

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.18s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.33s).
Accumulating evaluation results...
DONE (t=0.17s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.179
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.536
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.067
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.069
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.206
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.135
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.351
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

In [29]:
mAP50_list

[0.535649681927193,
 0.5080350658687016,
 0.5612143250249109,
 0.5373451512693791,
 0.5505163351805352]

In [30]:
np.mean(mAP50_list)

0.538552111854144

# YoloV5

In [31]:
SIZE = 512
IMG_SIZE = SIZE
size = SIZE

## Preparing Dataset

In [32]:
for fold in range(5):
    os.makedirs(f'/kaggle/tmp/dataset/{fold}')
    os.makedirs(f'/kaggle/tmp/dataset/{fold}/images')
    os.makedirs(f'/kaggle/tmp/dataset/{fold}/images/train')
    os.makedirs(f'/kaggle/tmp/dataset/{fold}/images/val')
    os.makedirs(f'/kaggle/tmp/dataset/{fold}/labels')
    os.makedirs(f'/kaggle/tmp/dataset/{fold}/labels/train')
    os.makedirs(f'/kaggle/tmp/dataset/{fold}/labels/val')

In [33]:
val_ann_file_list = [val_ann_file_0,val_ann_file_1,val_ann_file_2,val_ann_file_3,val_ann_file_4]
val_image_paths_list = []

for fold in range(5):
    val_ann_file = val_ann_file_list[fold]
    with open(val_ann_file,'rb') as file:
        val_dict = json.load(file)
    val_image_paths = []
    for item in tqdm(val_dict['images']):
        val_image_paths.append(item['file_name'])
    val_image_paths_list.append(val_image_paths)
    for image in tqdm(val_image_paths):
        image_id = image.split('_')[0]
        shutil.copyfile(os.path.join('/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/train_512x512',
                                 image),f'/kaggle/tmp/dataset/{fold}/images/val/{image_id}.png')

100%|██████████| 858/858 [00:01<00:00, 758.62it/s]


In [34]:
for fold in range(5):
    base_val_path = f'/kaggle/tmp/dataset/{fold}/images/val/'
    print(f'preprocessing images in folder: {base_val_path} ...')
    clahe = cv2.createCLAHE(clipLimit=2.0)
    for image in tqdm(os.listdir(base_val_path)):
        full_image_path = os.path.join(base_val_path,image)
        img = cv2.imread(full_image_path,cv2.IMREAD_UNCHANGED)
        img_eq = cv2.equalizeHist(img)
        img_clahe = clahe.apply(img_eq)
        cv2.imwrite(full_image_path,img_clahe)

preprocessing images in folder: /kaggle/tmp/dataset/0/images/val/ ...


100%|██████████| 859/859 [00:08<00:00, 99.20it/s]


preprocessing images in folder: /kaggle/tmp/dataset/1/images/val/ ...


100%|██████████| 859/859 [00:08<00:00, 97.09it/s] 


preprocessing images in folder: /kaggle/tmp/dataset/2/images/val/ ...


100%|██████████| 859/859 [00:08<00:00, 95.51it/s]


preprocessing images in folder: /kaggle/tmp/dataset/3/images/val/ ...


100%|██████████| 859/859 [00:10<00:00, 84.05it/s] 


preprocessing images in folder: /kaggle/tmp/dataset/4/images/val/ ...


100%|██████████| 858/858 [00:08<00:00, 97.05it/s] 


In [35]:
import yaml

for fold in range(5):
    data_yaml = dict(
        path = f'/kaggle/tmp/dataset/{fold}',
        train = 'images/train/',
        val = 'images/val/',
        nc=1,
        names = ['Covid_Opacity',]
    )

    with open(f'/kaggle/working/yolov5/data/data_fold{fold}.yaml','w') as file:
        yaml.dump(data_yaml,file,default_flow_style=True)

In [36]:
df = pd.read_csv('/kaggle/input/siim-covid19-detection/train_image_level.csv')
df['id'] = df.apply(lambda row: row.id.split('_')[0], axis=1)
df['image_level'] = df.apply(lambda row: row.label.split(' ')[0], axis=1)
meta_df = pd.read_csv(f'/kaggle/input/siim-covid-19-256-512-768-1024-1280/train_meta_{size}x{size}.csv')
meta_df['id'] = meta_df.apply(lambda row: row.id.split('_')[0], axis=1)
meta_df.drop('folder_id',inplace=True,axis=1)
meta_df.drop('study_id',inplace=True,axis=1)
meta_df['dim0'] = meta_df['height']
meta_df['dim1'] = meta_df['width']
meta_df.drop('width',inplace=True,axis=1)
meta_df.drop('height',inplace=True,axis=1)
df = df.merge(meta_df, on='id',how="left")

In [37]:
for fold in range(5):
    val_image_paths = val_image_paths_list[fold]
    for i,image in tqdm(enumerate(val_image_paths)):
        image_id = image.split('_')[0]
        row = df[df['id']==image_id].iloc[0]
        label = row.image_level
        if label=='opacity':
            bboxes = get_bbox(row)
            scale_bboxes = scale_bbox(row,bboxes)
            yolo_bboxes = get_yolo_format_bbox(SIZE,SIZE,scale_bboxes)
            with (open(f'/kaggle/tmp/dataset/{fold}/labels/val/{image_id}.txt','w')) as file:
                for bbox in yolo_bboxes:
                    bbox = [0]+bbox
                    bbox = [str(i) for i in bbox]
                    bbox = ' '.join(bbox)
                    file.write(bbox)
                    file.write('\n')

859it [00:01, 840.86it/s]
859it [00:01, 858.32it/s]
859it [00:01, 844.03it/s]
859it [00:01, 688.82it/s]
858it [00:01, 839.78it/s]


## Downloading YoloV5x Models

In [38]:
yoloV5_dowloaded_weights = download_wandb_models(run,'YoloV5x',['run_109ou5qr_model:best',
                                         'run_1jbxg1v5_model:best',
                                         'run_16d7mbhm_model:best',
                                         'run_31z85lql_model:best',
                                         'run_2eh81g2f_model:best'])

100%|██████████| 5/5 [00:37<00:00,  7.48s/it]


In [39]:
!python /kaggle/working/yolov5/val.py \
--weights /kaggle/tmp/downloaded_models/YoloV5x/fold_0_best.pt \
--data data_fold0.yaml\
--img 512\
--project /kaggle/working/results/yoloV5_fold_0\
--save-txt --save-conf

!python /kaggle/working/yolov5/val.py \
--weights /kaggle/tmp/downloaded_models/YoloV5x/fold_1_best.pt \
--data data_fold1.yaml\
--img 512\
--project /kaggle/working/results/yoloV5_fold_1\
--save-txt --save-conf

!python /kaggle/working/yolov5/val.py \
--weights /kaggle/tmp/downloaded_models/YoloV5x/fold_2_best.pt \
--data data_fold2.yaml\
--img 512\
--project /kaggle/working/results/yoloV5_fold_2\
--save-txt --save-conf

!python /kaggle/working/yolov5/val.py \
--weights /kaggle/tmp/downloaded_models/YoloV5x/fold_3_best.pt \
--data data_fold3.yaml\
--img 512\
--project /kaggle/working/results/yoloV5_fold_3\
--save-txt --save-conf

!python /kaggle/working/yolov5/val.py \
--weights /kaggle/tmp/downloaded_models/YoloV5x/fold_4_best.pt \
--data data_fold4.yaml\
--img 512\
--project /kaggle/working/results/yoloV5_fold_4\
--save-txt --save-conf

val: data=/kaggle/working/yolov5/data/data_fold0.yaml, weights=['/kaggle/tmp/downloaded_models/YoloV5x/fold_0_best.pt'], batch_size=32, imgsz=512, conf_thres=0.001, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=True, save_hybrid=False, save_conf=True, save_json=False, project=/kaggle/working/results/yoloV5_fold_0, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 23701ea Python-3.7.12 torch-1.9.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model summary: 444 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
100%|████████████████████████████████████████| 755k/755k [00:00<00:00, 24.3MB/s]
val: Scanning '/kaggle/tmp/dataset/0/labels/val' images and labels...859 found, 
val: New cache created: /kaggle/tmp/dataset/0/labels/val.cache
                 Class     Images  Instances          P          R     mAP@.5 mA
                   all        859       1580      0.579      0.541      0.539      0.173
Spe

In [40]:
id_translation_dict = {}
for fold in range(5):
    with open(f'/kaggle/working/coco_val_512x512_fold_{fold}.json','rb') as file:
        val_ann_json = json.load(file)
    for img_dict in val_ann_json['images']:
        img_name = img_dict['file_name'].split('_')[0]
        id_translation_dict[img_name] = img_dict['id']

In [41]:
def yolo_txt_labels_to_json(label_path,out_json_path,size):
    results = []
    for i,label in enumerate(os.listdir(label_path)):
        img_id = label.split('.')[0]
        img_id = id_translation_dict[img_id]
        with open(os.path.join(label_path,label),'r') as file:
            for line in file.readlines():
                line = line.strip()
                ann = line.split(' ')
                bbox = [float(x) for x in ann[1:5]]
                yolo_bbox = yolo_to_coco_bbox(size,bbox)
                conf = ann[5]
                result_dict = dict(
                    image_id=img_id,
                    bbox=yolo_bbox,
                    score=float(conf),
                    category_id=0)
                results.append(result_dict)
    with open(out_json_path,'w') as file:
        json.dump(results,file)
    return results   

In [42]:
!ls /kaggle/working/results

cascade_rcnn  retinanet      yoloV5_fold_1  yoloV5_fold_3
faster_rcnn   yoloV5_fold_0  yoloV5_fold_2  yoloV5_fold_4


In [43]:
yolo_results_json_path = ['/kaggle/working/results/yoloV5_fold_0/yoloV5_fold0.json',
                         '/kaggle/working/results/yoloV5_fold_1/yoloV5_fold1.json',
                         '/kaggle/working/results/yoloV5_fold_2/yoloV5_fold2.json',
                         '/kaggle/working/results/yoloV5_fold_3/yoloV5_fold3.json',
                         '/kaggle/working/results/yoloV5_fold_4/yoloV5_fold4.json',
                        ]

for fold in tqdm(range(5)):
    yolo_txt_labels_to_json(f'/kaggle/working/results/yoloV5_fold_{fold}/exp/labels/',yolo_results_json_path[fold],512)

100%|██████████| 5/5 [00:08<00:00,  1.71s/it]


In [44]:
yoloV5_mAP50_list = []
for fold in range(5):
    cocoGt = COCO(f'/kaggle/working/coco_val_512x512_fold_{fold}.json')
    cocoDt = cocoGt.loadRes(yolo_results_json_path[fold])
    imgIds=sorted(cocoGt.getImgIds())
    cocoEval = COCOeval(cocoGt,cocoDt,'bbox')

    cocoEval.params.catIds = [0] 
    cocoEval.evaluate()  
    cocoEval.accumulate()
    cocoEval.summarize()
    mAP50 = cocoEval.stats[1]
    yoloV5_mAP50_list.append(mAP50)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.75s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.99s).
Accumulating evaluation results...
DONE (t=0.73s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.174
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.537
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.064
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.050
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.202
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.128
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.341
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

In [45]:
np.mean(yoloV5_mAP50_list)

0.508508931955668

## Uploading Visual Predictions to WandB

run = wandb.init(project=PROJECT_NAME,
                 name=f'visualization_{EXP_NAME}',
                 job_type='images')

In [46]:
def upload_images_to_wandb(imagepaths,model,results_list):
    class_id_to_label = {
        1: "pred_covid_abnormality",
        2: "GT_covid_abnormality"
    }

    wnb_images = []

    for img_id, result in zip(imagepaths, results_list):

        image = cv2.imread(os.path.join(imgs_path, img_id))
        im_size = image.shape[0]
        bboxes = result[:, :4]
        scores = result[:, 4]
        ann_dict = {"predictions":{
                            "box_data":[],
                            "class_labels": class_id_to_label
                            },
                    "ground_truth":{
                            "box_data":[],
                            "class_labels": class_id_to_label
                            }
                        }

        for box, score in zip(bboxes, scores):
            single_data = {
                # one box expressed in the default relative/fractional domain
                "position": {
                    "minX": round(float(box[0])/im_size, 3),
                    "maxX": round(float(box[2])/im_size, 3),
                    "minY": round(float(box[1])/im_size, 3),
                    "maxY": round(float(box[3])/im_size, 3),
                },
                "class_id" : 1,
                "box_caption": class_id_to_label[1],
                "scores" : {
                    "confidence": float(score),
                }
            }
            ann_dict["predictions"]["box_data"].append(single_data)

        image_annotations = df_annotations[df_annotations.id==img_id.strip('.png')]

        for idxx, row in image_annotations[['xmin', 'ymin', 'xmax', 'ymax']].iterrows():
            single_data = {
                # one box expressed in the default relative/fractional domain
                "position": {
                    "minX": round(float(row[0])/im_size, 3),
                    "maxX": round(float(row[2])/im_size, 3),
                    "minY": round(float(row[1])/im_size, 3),
                    "maxY": round(float(row[3])/im_size, 3),
                },
                "class_id" : 2,# Class id 2 is used to denote ground truth
                "box_caption": class_id_to_label[2],
                "scores" : {
                    "confidence": 1.0,
                }
            }
            ann_dict["ground_truth"]["box_data"].append(single_data)

        wnb_images.append(wandb.Image(image, boxes=ann_dict,caption=img_id))

    wandb.log({f'images-{model}': wnb_images})



imgs_path = '/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/train_512x512'
with open(val_ann_file) as file:
    val_ann = json.load(file)
imagepaths = [item['file_name'] for item in val_ann['images'][:]]

df_annotations = pd.read_csv('/kaggle/input/siim-covid19-512-images-and-metadata/df_train_processed_meta.csv')

results_list = []

for idx, img_id in tqdm(enumerate(imagepaths)):
    img_path = os.path.join(imgs_path, img_id)
    img = cv2.imread(img_path)
    result = inference_detector(retinanet, img_path)
    results_list.append(result[0])

upload_images_to_wandb(imagepaths,'RetinaNet',results_list)

for idx, img_id in tqdm(enumerate(imagepaths)):
    img_path = os.path.join(imgs_path, img_id)
    img = cv2.imread(img_path)
    result = inference_detector(cascade_rcnn, img_path)
    results_list.append(result[0])

upload_images_to_wandb(imagepaths,'Cascade RCNN',results_list)


In [47]:
run.finish()